# מדריך לקריאה מקבילית של פונקציות עם Phi-4 Mini ONNX

מחברת זו מדגימה כיצד להשתמש ב-Phi-4 Mini עם ONNX Runtime GenAI לקריאה מקבילית של פונקציות. קריאה לפונקציות מאפשרת למודל להפעיל בצורה חכמה כלים חיצוניים ו-APIs בהתאם לבקשות המשתמש.

## סקירה כללית

במדריך זה תלמדו כיצד:
- להגדיר את Phi-4 Mini עם ONNX Runtime GenAI
- להגדיר סכמות פונקציות להזמנת טיסות ובתי מלון
- להשתמש ביצירה מונחית עם דקדוק Lark לתוצאות מובנות
- לבצע קריאות מקבילות לפונקציות בתרחישים מורכבים של הזמנת נסיעות

## דרישות מקדימות

לפני הפעלת המחברת, ודאו שיש לכם:
- הורדתם את מודל Phi-4 Mini ONNX
- התקנתם את חבילת `onnxruntime-genai`
- הבנה בסיסית של מושגי קריאה לפונקציות


## שלב 1: ייבוא ספריות נדרשות

ראשית, נייבא את הספריות הנחוצות למימוש קריאת הפונקציות שלנו.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## שלב 2: הגדרת המודל ותצורה

עכשיו נגדיר את מודל Phi-4 Mini ONNX. ודא להחליף את הנתיב עם ספריית המודל האמיתית שלך.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## שלב 3: הגדרת פרמטרי יצירה

הגדר את פרמטרי היצירה כדי לשלוט בהתנהגות הפלט של המודל. הגדרות אלו מבטיחות תגובות ממוקדות ודטרמיניסטיות לקריאה לפונקציות.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## שלב 4: הגדרת פונקציות זמינות

כאן אנו מגדירים את הפונקציות שהעוזר האינטליגנטי שלנו יכול לקרוא להן. בדוגמה זו, יש לנו שתי פונקציות הקשורות לנסיעות:
1. **booking_flight_tickets**: להזמנת טיסות בין שדות תעופה
2. **booking_hotels**: להזמנת מקומות לינה בבתי מלון

הגדרות הפונקציות עוקבות אחר פורמט סכמת קריאת הפונקציות של OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## שלב 5: פונקציות עזר ליצירת דקדוק

פונקציות העזר הללו ממירות את הגדרות הפונקציות שלנו לפורמט דקדוק של Lark, שבו משתמש ONNX Runtime GenAI לצורך יצירה מונחית. זה מבטיח שהמודל יפיק קריאות פונקציה תקינות בפורמט JSON הנכון.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## שלב 6: בדיקת יצירת דקדוק

בואו נבדוק את פונקציות יצירת הדקדוק שלנו כדי לראות איך הן ממירות את הגדרות הכלים שלנו לפורמט המתאים.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## שלב 7: הכנת הנחיית מערכת ומחולל

עכשיו נכין את הנחיית המערכת שמסבירה למודל על הכלים הזמינים ונגדיר את המחולל עם פרמטרים להנחיית יצירה.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## שלב 8: אתחול הגנרטור עם יצירה מונחית

עכשיו ניצור את הגנרטור עם הפרמטרים שהגדרנו וניישם את דקדוק Lark ליצירה מונחית.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## שלב 9: בדיקת קריאה מקבילה לפונקציות

עכשיו נבדוק את ההגדרות שלנו עם תרחיש מורכב של הזמנת נסיעות שדורש קריאה למספר פונקציות.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### יצירת קריאות לפונקציות

המודל ייצור כעת קריאות לפונקציות מובנות בהתבסס על בקשת המשתמש. בזכות יצירה מונחית, הפלט יהיה בפורמט JSON תקין שניתן להריץ ישירות.

**פלט צפוי**: המודל צריך ליצור קריאות לפונקציות עבור:
1. `booking_flight_tickets` עם פרטי טיסה מבייג'ין (PEK) לפריז (CDG)
2. `booking_hotels` עם פרטי לינה בפריז

הרץ את התא למטה כדי לראות את היצירה בזמן אמת:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## סיכום

🎉 **מזל טוב!** הצלחת ליישם קריאה מקבילה לפונקציות עם Phi-4 Mini באמצעות ONNX Runtime GenAI.

### מה שלמדת:

1. **הגדרת מודל**: איך להגדיר את Phi-4 Mini עם ONNX Runtime GenAI  
2. **הגדרת פונקציות**: איך להגדיר סכמות פונקציות לקריאה לפונקציות AI  
3. **יצירה מונחית**: איך להשתמש בדקדוק Lark ליצירת פלט מובנה  
4. **קריאות פונקציות מקבילות**: איך להתמודד עם בקשות מורכבות הדורשות קריאות פונקציות מרובות  

### יתרונות מרכזיים:

- ✅ **פלט מובנה**: יצירה מונחית מבטיחה קריאות פונקציות JSON תקינות  
- ✅ **עיבוד מקבילי**: טיפול בקריאות פונקציות מרובות בבקשה אחת  
- ✅ **ביצועים גבוהים**: ONNX Runtime מספק הסקה אופטימלית  
- ✅ **סכמה גמישה**: קל להוסיף או לשנות הגדרות פונקציות  

### משאבים:

- [תיעוד Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [תיעוד ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)  
- [שיטות עבודה מומלצות לקריאה לפונקציות](https://platform.openai.com/docs/guides/function-calling)  



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
